# MET Data Processing to FACEMDS Format
=============================
by Bharat Sharma and Anthony Walker

## Requisites
The processed files from the plot data are saved at `/Users/ud4/Documents/FACEMDS/MET_Data_Processing/Oren_2022_Met_Data_processed/` <br>
There are two files: <br>
1. `Processed_Duke_Met_Data_All_Vars_30m_FV.csv`: FillValue/Missing = -6999.0
2. `Processed_Duke_Met_Data_All_Vars_30m.csv`: FillValue/Missing = NaN or empty

In [1]:
# importing libraries
import xarray as xr
import glob
from datetime import datetime
import cftime
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import xarray as xr

In [28]:
def PAR2SWdown (data):
    """
    
    """
    conversion_factor = 0.47
    return data*conversion_factor
    

In [2]:
# paths

paths = {}
paths ["ELM-DUKE"] = "/Users/ud4/Documents/FACEMDS/MET_Data_Processing/ELM_Data/data/atm/datm7/CLM1PT_data/1x1pt_US-DUK/"
paths ["FACEMDS_Walker2018"] = "/Users/ud4/Documents/FACEMDS/MET_Data_Processing/Walker_2018_FATES_MDS/data/"
paths ["DukeFACE_Oren2022"] = "/Users/ud4/Documents/FACEMDS/MET_Data_Processing/Oren_2022_DUKE_Met/data/"
paths ["Save_Processed"] = "/Users/ud4/Documents/FACEMDS/MET_Data_Processing/Oren_2022_Met_Data_processed/"

In [3]:
# Read the Processed Data
df_all_vars_30m_FV = pd.read_csv(f"{paths['Save_Processed']}Processed_Duke_Met_Data_All_Vars_30m_FV.csv")
df_all_vars_30m = pd.read_csv(f"{paths['Save_Processed']}Processed_Duke_Met_Data_All_Vars_30m.csv")

## Creating DUKE_forcing_h.txt

In [4]:
# Dictionary of columns and their descriptions
dict_cols = {
'YEAR':'Year of measurement',
'DTIME':'Fractional day of year',
'DOY':'Day of year',
'HRMIN':'Hour:minute, marked at the middle of measurement interval with last two digits as minute',
'Rainf':'Total Precipitation over a time step of measurement',
'Rainf_f': 'gap-filling flag, 0 = measured, 1 = derived from other variables, 2 = filled by \
mean diurnal cycle within 5-15 days, 3 = filled by data from nearby weather station, 4 = \
filled by using NARR (North American Regional Reanalysis) data',
'Tair':'Mean air temperature over a time step of measurement',
'Tair_f':'gap-filling flag',
'RH':'Mean relative humidity over a time step of measurement',
'RH_f':'gap-filling flag',
'VPD':'Vapor pressure deficit, kPa',
'VPD_f':'gap-filling flag',
'PAR':'Incident or downward photosynthetically active radiation',   
'PAR_f':'gap-filling flag',    
'SM':'Soil Moisture integrates measurements from 0 to 30cm depth',
'SM_f':'gap-filling flag', 
'SWP':' Soil Water Potential',
'SWP_f':'gap-filling flag', 
'SVP':'Saturated Vapor Pressure',
'SVP_f':'gap-filling flag', 
'Rn':'Net Radiation',
'Rn_f':'gap-filling flag', 
'SLT':'Soil Temperature',   
'SLT_f':'gap-filling flag', 
}

In [14]:
# Units of Saving File Format
# Change: Rainf, mm to kg/m2/s; factor_add = 0,  factor_multiple = 1;  
# Change: Tair,  deg C to K   ; factor_add = 273.15,  factor_multiple = 1;
# Change: RH,    '' to %      ; factor_add = 0,  factor_multiple = 100;
# Change: VPD,   kPa to Pa    ; factor_add = 0,  factor_multiple = 1000;
# Change: SLT,    deg C to K   ; factor_add = 273.15,  factor_multiple = 1;
dict_units = {
    'Rainf':'kg/m2/s', # 'mm' = 'kg/m2/s'
    'Tair':'K',
    'RH':'%',
    'VPD':'Pa',
    'PAR':'umol/m2/s',
    'SM':'',
    'SWP':'',
    'SVP':'kPa',
    'Rn':'umol/m2/s',
    'SLT':'K'
}



In [ ]:
df_h = pd.DataFrame(columns=dict_cols.keys())
df_h['YEAR'] = df_all_vars_30m['Year'].astype(int)
df_h['DOY'] = df_all_vars_30m['DOY'].astype(int)
df_h['HRMIN'] = df_all_vars_30m['Time'].astype(int)
df_h['DTIME'] = (df_all_vars_30m['Time']/(24*60)+df_all_vars_30m['DOY']).astype(float)
df_h[list(dict_units.keys())] = df_all_vars_30m[list(dict_units.keys())]
gap_fill_cols = [col + '_f' for col in list(dict_units.keys())]
df_h[gap_fill_cols] = 0

#DROPING THE LAST ROW
df_h.drop(df_h.index[-1], inplace=True)
for k_var in ['Rainf']:
    factor_add = 0
    factor_multiple = 1/(60*30) # From per day to per second for 48 timesteps in a day i.e. 24*60*60/48
    df_h[k_var] = df_h[k_var]*factor_multiple+factor_add
for k_var in ['Tair','SLT']:
    factor_add = 273.15
    factor_multiple = 1
    df_h[k_var] = df_h[k_var]*factor_multiple+factor_add
for k_var in ['RH']:
    factor_add = 0
    factor_multiple = 100
    df_h[k_var] = df_h[k_var]*factor_multiple+factor_add
for k_var in ['VPD']:
    factor_add = 0
    factor_multiple = 1000
    df_h[k_var] = df_h[k_var]*factor_multiple+factor_add
# Adding the unit row below first row
unit_row = pd.Series([dict_units.get(col, '') for col in df_h.columns], index=df_h.columns)
df_h_save = pd.concat([pd.DataFrame([unit_row]), df_h.iloc[:]]).reset_index(drop=True)

df_h_save.to_csv(f"{paths['Save_Processed']}DUKE_forcing_h.csv")
df_h_save.to_csv(f"{paths['Save_Processed']}DUKE_forcing_h.txt")
fill_value = -6999.
df_h_fv_save = df_h_save.fillna(fill_value)
df_h_fv_save.to_csv(f"{paths['Save_Processed']}DUKE_forcing_h_fv.csv")
df_h_fv_save.to_csv(f"{paths['Save_Processed']}DUKE_forcing_h_fv.txt")

### With Time Index

In [ ]:
df_h_wTime = df_h.copy(deep=True)
df_h_wTime['Time'] = df_all_vars_30m['Date']
df_h_wTime['Time'] = pd.to_datetime(df_h_wTime['Time'])
df_h_wTime = df_h_wTime.set_index('Time')
df_h_wTime

,YEAR,DTIME,DOY,HRMIN,Rainf,Rainf_f,Tair,Tair_f,RH,RH_f,...,SM,SM_f,SWP,SWP_f,SVP,SVP_f,Rn,Rn_f,SLT,SLT_f
Time,,,,,,,,,,,,,,,,,,,,,
1997-01-01 00:30:00,1997,1.020833,1,30,NaN,0,NaN,0,NaN,0,...,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0
1997-01-01 01:00:00,1997,1.069444,1,100,NaN,0,NaN,0,NaN,0,...,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0
1997-01-01 01:30:00,1997,1.090278,1,130,NaN,0,NaN,0,NaN,0,...,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0
1997-01-01 02:00:00,1997,1.138889,1,200,NaN,0,NaN,0,NaN,0,...,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0
1997-01-01 02:30:00,1997,1.159722,1,230,NaN,0,NaN,0,NaN,0,...,NaN,0,NaN,0,NaN,0,NaN,0,NaN,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2012-12-31 21:30:00,2012,367.479167,366,2130,0.0,0,281.13,0,42.0,0,...,0.39,0,488.93,0,1.07,0,-43.54,0,283.55,0
2012-12-31 22:00:00,2012,367.527778,366,2200,0.0,0,280.73,0,44.0,0,...,0.39,0,488.87,0,1.04,0,-44.50,0,283.55,0
2012-12-31 22:30:00,2012,367.548611,366,2230,0.0,0,280.23,0,46.0,0,...,0.39,0,488.87,0,1.01,0,-41.98,0,283.55,0


## Creating DUKE_forcing_d.txt
by talking average across all variables except <br>
PAR, Rn, and Rainf where we computed the sum, resulting in Total daily metrics

In [17]:
# Units of Saving File Format
# Change: Rainf, kg/m2/s to 'kg/m2/d'; factor_add = 0,  factor_multiple = 30*60 (b/c 24*60*60/48);  
# Change: PAR/Rn, umol/m2/s to 'mol/m2/d'; factor_add = 0,  factor_multiple = 30*60 *10(--6);  
dict_units_d = {
    'Rainf':'kg/m2/d', # 'kg/m2/s' to 'kg/m2/d'
    'Tair':'K',
    'RH':'%',
    'VPD':'Pa',
    'PAR':'mol/m2/d',
    'SM':'',
    'SWP':'',
    'SVP':'kPa',
    'Rn':'mol/m2/d',
    'SLT':'K'
}

In [18]:
# Using the datetime index to calculate means
df_d_wTime = df_h_wTime.resample('D').mean()
df_d_wTime= df_d_wTime.drop(['DTIME','HRMIN'], axis=1)
df_d_wTime['DOY'] = round(df_d_wTime['DOY']).astype('int')

# For Variables to convert from per second to per day if the original Tstep is of 30 mins
factor_multiple_s2d = 30*60

# For Rainf we need to take sum
df_d_wTime['Rainf'] = df_h_wTime[['YEAR', 'DOY', 'Rainf']].resample('D').sum()['Rainf'] * factor_multiple_s2d
# For PAR we need to take sum
df_d_wTime['PAR'] = df_h_wTime[['YEAR', 'DOY', 'PAR']].resample('D').sum()['PAR'] * factor_multiple_s2d *10**(-6)
# For Rn we need to take sum
df_d_wTime['Rn'] = df_h_wTime[['YEAR', 'DOY', 'Rn']].resample('D').sum()['Rn'] * factor_multiple_s2d *10**(-6)

df_d_wTime['YEAR'] =  df_d_wTime['YEAR'].astype('int')

# Reset index to columns
df_d = df_d_wTime.reset_index()
df_d = df_d.drop('Time', axis =1)

# Adding the unit row below first row
unit_row = pd.Series([dict_units_d.get(col, '') for col in df_d.columns], index=df_d.columns)
df_d_save = pd.concat([pd.DataFrame([unit_row]), df_d.iloc[:]]).reset_index(drop=True)

#Saving the dataframes
df_d_save.to_csv(f"{paths['Save_Processed']}DUKE_forcing_d.csv")
df_d_save.to_csv(f"{paths['Save_Processed']}DUKE_forcing_d.txt")
fill_value = -6999.
df_d_fv_save = df_d_save.fillna(fill_value)
df_d_fv_save.to_csv(f"{paths['Save_Processed']}DUKE_forcing_d_fv.csv")
df_d_fv_save.to_csv(f"{paths['Save_Processed']}DUKE_forcing_d_fv.txt")


## Creating DUKE_forcing_y.txt




In [25]:
# Units of Saving File Format
# Change: Rainf, kg/m2/d to 'kg/m2/y'; ;  
dict_units_y = {
    'Rainf':'kg/m2/y', # 'kg/m2/d' to 'kg/m2/y'
    'Tair':'K',
    'RH':'%',
    'VPD':'Pa',
    'PAR':'mol/m2/y',
    'SM':'',
    'SWP':'',
    'SVP':'kPa',
    'Rn':'mol/m2/y',
    'SLT':'K'
}

In [26]:
# Using the datetime index to calculate means
df_y_wTime = df_d_wTime.resample('Y').mean()
df_y_wTime= df_y_wTime.drop('DOY', axis=1)
df_y_wTime['YEAR'] = round(df_y_wTime['YEAR']).astype('int')

# For Rainf we need to take sum
df_y_wTime['Rainf'] = df_d_wTime[['YEAR', 'Rainf']].resample('Y').sum()['Rainf']
df_y_wTime['YEAR'] =  df_y_wTime['YEAR'].astype('int')

# Reset index to columns
df_y = df_y_wTime.reset_index()
df_y = df_y.drop('Time', axis =1)

# Adding the unit row below first row
unit_row = pd.Series([dict_units_y.get(col, '') for col in df_y.columns], index=df_y.columns)
df_y_save = pd.concat([pd.DataFrame([unit_row]), df_y.iloc[:]]).reset_index(drop=True)

#Saving the dataframes
df_y_save.to_csv(f"{paths['Save_Processed']}DUKE_forcing_y.csv")
df_y_save.to_csv(f"{paths['Save_Processed']}DUKE_forcing_y.txt")
fill_value = -6999.
df_y_fv_save = df_y_save.fillna(fill_value)
df_y_fv_save.to_csv(f"{paths['Save_Processed']}DUKE_forcing_y_fv.csv")
df_y_fv_save.to_csv(f"{paths['Save_Processed']}DUKE_forcing_y_fv.txt")

In [27]:
df_y_fv_save

,YEAR,Rainf,Rainf_f,Tair,Tair_f,RH,RH_f,VPD,VPD_f,PAR,...,SM,SM_f,SWP,SWP_f,SVP,SVP_f,Rn,Rn_f,SLT,SLT_f
0,,kg/m2/y,,K,,%,,kPa,,mol/m2/y,...,,,,,kPa,,mol/m2/y,,K,
1,1997,636.72,0.0,289.42821,0.0,76.803829,0.0,587.834211,0.0,29.208546,...,0.281872,0.0,-6999.0,0.0,2.099477,0.0,0.0,0.0,289.116283,0.0
2,1998,1209.38,0.0,288.725132,0.0,76.778425,0.0,571.555936,0.0,30.392428,...,0.297116,0.0,-6999.0,0.0,2.045817,0.0,9.740487,0.0,287.719291,0.0
3,1999,1363.01,0.0,288.087789,0.0,72.841495,0.0,571.615297,0.0,30.576171,...,0.345106,0.0,-6999.0,0.0,1.956654,0.0,8.989835,0.0,287.67464,0.0
4,2000,1131.52,0.0,287.147828,0.0,75.634904,0.0,454.773452,0.0,27.781377,...,0.314247,0.0,-6999.0,0.0,1.861967,0.0,8.8055,0.0,287.311951,0.0
5,2001,946.86,0.0,287.825286,0.0,74.564384,0.0,488.350457,0.0,31.041523,...,0.249167,0.0,-6999.0,0.0,1.902035,0.0,9.122238,0.0,287.564886,0.0
6,2002,1092.2,0.0,288.397873,0.0,72.894806,0.0,613.063356,0.0,28.997837,...,0.283562,0.0,-6999.0,0.0,2.023352,0.0,8.7408,0.0,288.13946,0.0
7,2003,1345.86,0.0,287.475367,0.0,78.422374,0.0,391.248288,0.0,27.202611,...,0.350841,0.0,-6999.0,0.0,1.877743,0.0,8.049955,0.0,287.809599,0.0
8,2004,991.96,0.0,287.984917,0.0,77.523167,0.0,423.266166,0.0,28.549687,...,0.335384,0.0,-6999.0,0.0,1.937918,0.0,7.966916,0.0,288.148327,0.0
9,2005,934.4,0.0,287.985529,0.0,77.675457,0.0,443.706621,0.0,27.970881,...,0.292478,0.0,-6999.0,0.0,1.964302,0.0,8.07191,0.0,287.950119,0.0


# Making netcdf files


## Creating DUKE_forcing_h.nc

In [10]:
cols_netcdf_h = ['YEAR',
 'DOY',
 'HRMIN',
 'Rainf',
  'Tair',
  'RH',
  'VPD',
  'PAR',
  'SM',
  'SWP',
  'SVP',
  'Rn',
  'SLT']

# Convert the DataFrame to an xarray Dataset

df_h_wTime_fv = df_h_wTime.fillna(fill_value)
ds = xr.Dataset.from_dataframe(df_h_wTime_fv[cols_netcdf_h])
ds = ds.expand_dims({'lat': 1}).assign_coords({'lat':[35.9782] })
ds = ds.expand_dims({'lon': 1}).assign_coords({'lon':[-79.0942] })
ds['lon'].attrs['units'] = "degrees_east"
ds['lon'].attrs['long_name'] = "Longitude"
ds['lat'].attrs['units'] = "degrees_north"
ds['lat'].attrs['long_name'] = "Latitude"
ds['YEAR'].attrs['units'] = ""
ds['YEAR'].attrs['long_name'] = "Year of Measurement"
ds['DOY'].attrs['units'] = ""
ds['DOY'].attrs['long_name'] = "Day of Measurement"
ds['HRMIN'].attrs['units'] = ""
ds['HRMIN'].attrs['long_name'] = "Hour:minute - marked at the middle of measurement interval with last two digits as minute"
# Adding attributes to variables
for k_var in cols_netcdf_h[-10:]: # looping of last 10 columns
    ds[k_var].attrs['units'] = dict_units[k_var]
    ds[k_var].attrs['missing_value'] = fill_value
    ds[k_var].attrs['long_name'] = dict_cols[k_var]
    ds[k_var].attrs['associate'] = "Time lat lon"
    ds[k_var].attrs['axis'] = "TYX"

    
    
# Add global attributes to the Dataset
ds.attrs['site_id'] = "DUKE"
ds.attrs['title'] = "Half-hourly forcing data from DUKE Forest FACE, North Carolina, USA"
ds.attrs['history'] = "File Origin - This file was created at Oak Ridge National Laboratory for FACE model data synthesis"
ds.attrs['creation_date'] = "Aug 24, 2023" ;
ds.attrs['contact'] = 'Bharat Sharma (sharmabd@ornl.gov), Anthony Walker (walkerp@ornl.gov)'
ds.to_netcdf(f"{paths['Save_Processed']}DUKE_forcing_h.nc")

print(f"Dataset saved to {paths['Save_Processed']}DUKE_forcing_h.nc")

Dataset saved to /Users/ud4/Documents/FACEMDS/MET_Data_Processing/Oren_2022_Met_Data_processed/DUKE_forcing_h.nc


## Creating DUKE_forcing_d.nc

In [11]:
cols_netcdf_d = ['YEAR',
 'DOY',
 'Rainf',
  'Tair',
  'RH',
  'VPD',
  'PAR',
  'SM',
  'SWP',
  'SVP',
  'Rn',
  'SLT']

# Convert the DataFrame to an xarray Dataset
df_d_wTime_fv = df_d_wTime.fillna(fill_value)
ds = xr.Dataset.from_dataframe(df_d_wTime_fv[cols_netcdf_d])
ds = ds.expand_dims({'lat': 1}).assign_coords({'lat':[35.9782] })
ds = ds.expand_dims({'lon': 1}).assign_coords({'lon':[-79.0942] })
ds['lon'].attrs['units'] = "degrees_east"
ds['lon'].attrs['long_name'] = "Longitude"
ds['lat'].attrs['units'] = "degrees_north"
ds['lat'].attrs['long_name'] = "Latitude"
ds['YEAR'].attrs['units'] = ""
ds['YEAR'].attrs['long_name'] = "Year of Measurement"
ds['DOY'].attrs['units'] = ""
ds['DOY'].attrs['long_name'] = "Day of Measurement"
# Adding attributes to variables
for k_var in cols_netcdf_h[-10:]: # looping of last 10 columns
    if True:
        ds[k_var].attrs['units'] = dict_units_d[k_var]
        ds[k_var].attrs['missing_value'] = fill_value
        ds[k_var].attrs['long_name'] = dict_cols[k_var]
        ds[k_var].attrs['associate'] = "Time lat lon"
        ds[k_var].attrs['axis'] = "TYX"

# Add global attributes to the Dataset
ds.attrs['site_id'] = "DUKE"
ds.attrs['title'] = "Daily forcing data from DUKE Forest FACE, North Carolina, USA"
ds.attrs['history'] = "File Origin - This file was created at Oak Ridge National Laboratory for FACE model data synthesis"
ds.attrs['creation_date'] = "Aug 24, 2023" ;
ds.attrs['contact'] = 'Bharat Sharma (sharmabd@ornl.gov), Anthony Walker (walkerp@ornl.gov)'
ds.to_netcdf(f"{paths['Save_Processed']}DUKE_forcing_d.nc")

print(f"Dataset saved to {paths['Save_Processed']}DUKE_forcing_d.nc")

Dataset saved to /Users/ud4/Documents/FACEMDS/MET_Data_Processing/Oren_2022_Met_Data_processed/DUKE_forcing_d.nc


## Creating DUKE_forcing_y.nc

In [12]:
cols_netcdf_y = ['YEAR',
 'Rainf',
  'Tair',
  'RH',
  'VPD',
  'PAR',
  'SM',
  'SWP',
  'SVP',
  'Rn',
  'SLT']

# Convert the DataFrame to an xarray Dataset
df_y_wTime_fv = df_y_wTime.fillna(fill_value)
ds = xr.Dataset.from_dataframe(df_y_wTime_fv[cols_netcdf_y])
ds = ds.expand_dims({'lat': 1}).assign_coords({'lat':[35.9782] })
ds = ds.expand_dims({'lon': 1}).assign_coords({'lon':[-79.0942] })
ds['lon'].attrs['units'] = "degrees_east"
ds['lon'].attrs['long_name'] = "Longitude"
ds['lat'].attrs['units'] = "degrees_north"
ds['lat'].attrs['long_name'] = "Latitude"
ds['YEAR'].attrs['units'] = ""
ds['YEAR'].attrs['long_name'] = "Year of Measurement"
# Adding attributes to variables
for k_var in cols_netcdf_h[-10:]: # looping of last 10 columns
    if k_var in ['PAR','Rn']:
        ds[k_var].attrs['units'] = dict_units_y[k_var]
        ds[k_var].attrs['missing_value'] = fill_value
        ds[k_var].attrs['long_name'] = f"Mean {dict_cols[k_var]}"
        ds[k_var].attrs['associate'] = "Time lat lon"
        ds[k_var].attrs['axis'] = "TYX"        
    else:
        ds[k_var].attrs['units'] = dict_units_y[k_var]
        ds[k_var].attrs['missing_value'] = fill_value
        ds[k_var].attrs['long_name'] = dict_cols[k_var]
        ds[k_var].attrs['associate'] = "Time lat lon"
        ds[k_var].attrs['axis'] = "TYX"

# Add global attributes to the Dataset
ds.attrs['site_id'] = "DUKE"
ds.attrs['title'] = "Yearly forcing data from DUKE Forest FACE, North Carolina, USA"
ds.attrs['history'] = "File Origin - This file was created at Oak Ridge National Laboratory for FACE model data synthesis"
ds.attrs['creation_date'] = "Aug 24, 2023" ;
ds.attrs['contact'] = 'Bharat Sharma (sharmabd@ornl.gov), Anthony Walker (walkerp@ornl.gov)'
ds.to_netcdf(f"{paths['Save_Processed']}DUKE_forcing_y.nc")

print(f"Dataset saved to {paths['Save_Processed']}DUKE_forcing_y.nc")

Dataset saved to /Users/ud4/Documents/FACEMDS/MET_Data_Processing/Oren_2022_Met_Data_processed/DUKE_forcing_y.nc


In [13]:
ds_walker_h = xr.open_dataset(f"{paths['FACEMDS_Walker2018']}DUKE_forcing_h.nc",decode_times=False)
ds_walker_h

<xarray.Dataset>
Dimensions:         (y: 1, x: 1, tstep: 210384)
Dimensions without coordinates: y, x, tstep
Data variables: (12/23)
    nav_lat         (y) float32 ...
    nav_lon         (x) float32 ...
    YEAR            (tstep) int32 ...
    DOY             (tstep) int32 ...
    HRMIN           (tstep) int32 ...
    TIME            (tstep) int32 ...
    ...              ...
    aCO2            (tstep, y, x) float32 ...
    eCO2            (tstep, y, x) float32 ...
    aO3             (tstep, y, x) float32 ...
    eO3             (tstep, y, x) float32 ...
    Ndep            (tstep, y, x) float32 ...
    SolarElevation  (tstep, y, x) float32 ...
Attributes:
    site_id:        DUKE
    title:          Half-hourly forcing data from DUKE Forest FACE, North Car...
    history:        File Origin - This file was created at Oak Ridge National...
    creation_date:  Oct 16, 2014
    contact:        Anthony Walker (walkerap@ornl.gov), Bai Yang (yangb@ornl....